In [1]:

import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), r'C:\Users\Joel\Desktop\Sprints Case Study'))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils.connectors import API_Client
from utils.transformations import remove_invalid_rows, convert_to_dataframe, convert_currency, annual_aggregation
from datetime import datetime
import pandas as pd

base_url = "https://technical-case-platform-engineer.onrender.com"

exchange_rate_endpoint = "exchange-rates"
monthly_date_endpoint = "monthly-data"
annual_data_endpoint = "annual-data"

today = datetime.today()

In [2]:
api_connector = API_Client(base_url)

currency_data = api_connector.get(exchange_rate_endpoint)
value_data = api_connector.get(monthly_date_endpoint)

currency_df = convert_to_dataframe(currency_data)
value_df = convert_to_dataframe(value_data)

In [3]:
cleaned_value_df = remove_invalid_rows(value_df, currency_df)
cleaned_value_df

,timestamp,value,company,currency
1,2025-01-01,6512.17,Solvex Solutions,SEK
2,2024-12-01,5394.51,Solvex Solutions,SEK
3,2024-11-01,3040.15,Solvex Solutions,SEK
4,2024-10-01,6558.26,Solvex Solutions,EUR
5,2024-09-01,5402.61,Solvex Solutions,SEK
...,...,...,...,...
428,2013-06-01,9090.43,VertexSphere,USD
429,2013-05-01,6123.12,VertexSphere,USD
430,2013-04-01,4246.13,VertexSphere,USD
431,2013-03-01,1381.20,VertexSphere,EUR


In [5]:
converted_df = convert_currency(cleaned_value_df, currency_df)
converted_df

,timestamp,value,company,currency
1,2025-01-01,6512.1700,Solvex Solutions,SEK
2,2024-12-01,5394.5100,Solvex Solutions,SEK
3,2024-11-01,3040.1500,Solvex Solutions,SEK
4,2024-10-01,75419.9900,Solvex Solutions,SEK
5,2024-09-01,5402.6100,Solvex Solutions,SEK
...,...,...,...,...
428,2013-06-01,102540.0504,VertexSphere,SEK
429,2013-05-01,69068.7936,VertexSphere,SEK
430,2013-04-01,47896.3464,VertexSphere,SEK
431,2013-03-01,15883.8000,VertexSphere,SEK


In [9]:
monthly_max_values = converted_df.groupby(["timestamp", "currency"])["value"].max().reset_index()

monthly_max_df = pd.merge(
    monthly_max_values, 
    converted_df, 
    on=["timestamp", "currency", "value"], 
    how="left"
)

monthly_max_df.to_csv("monthly-max-values.csv")

In [12]:
nexara_filtered = converted_df[converted_df["company"]=="Nexara Technologies"]
nexara_total = nexara_filtered.groupby(["company", "currency"]).agg(
    value = ("value", "sum")
).reset_index()

nexara_total.to_csv("nexara_total.csv")

In [16]:
annual_agg = annual_aggregation(converted_df)
records = annual_agg.to_dict(orient='records')
payload = {"data": records}
payload

{'data': [{'company': 'Nexara Technologies',
   'year': 2013,
   'currency': 'SEK',
   'value': 56254.68637999999},
  {'company': 'Nexara Technologies',
   'year': 2014,
   'currency': 'SEK',
   'value': 55221.31435},
  {'company': 'Nexara Technologies',
   'year': 2015,
   'currency': 'SEK',
   'value': 56392.964327272726},
  {'company': 'Nexara Technologies',
   'year': 2016,
   'currency': 'SEK',
   'value': 59863.99985},
  {'company': 'Nexara Technologies',
   'year': 2017,
   'currency': 'SEK',
   'value': 40127.55971666666},
  {'company': 'Nexara Technologies',
   'year': 2018,
   'currency': 'SEK',
   'value': 48112.668333333335},
  {'company': 'Nexara Technologies',
   'year': 2019,
   'currency': 'SEK',
   'value': 49536.77953333333},
  {'company': 'Nexara Technologies',
   'year': 2020,
   'currency': 'SEK',
   'value': 60533.85566},
  {'company': 'Nexara Technologies',
   'year': 2021,
   'currency': 'SEK',
   'value': 46364.30561666666},
  {'company': 'Nexara Technologies',